AQUI SE TRABAJA EL Analisis de data exploratorio (EDA)
- josu and gotzon

# PROBLEM DEFINITION

result comment

# Data Comprehension

data importation

In [ ]:
#data import

result comment

data inspection

In [1]:
#data inspection

result comment

data show

In [ ]:
#show data

result comment

# Data Cleaning

missing values

In [ ]:
#missing values

result comments

atipic values 

In [ ]:
#atipic values 

result comments

inconsistent data corrections

In [ ]:
#inconsistent data corrections

result comments

# Data Transformation

data type conversion

In [ ]:
#data type conversion

result comment

data normalization and standardization

In [ ]:
#data normalization and standardization

result comment

new variables creation

In [ ]:
#new variables creation

result comments

# Uni, Bi and multivaraint analisis

relation between variants

In [ ]:
#relation between variants

PCA? no

result comments

# Data Visualization

In [ ]:
#data visualization

result comments

CONCLUSSIONS